In [1]:
# 2020 Q2부터 시작할 수 있음..

high_list = [
    "4. 재무제표.csv",
    "2. 연결재무제표.csv",
    "1. 요약재무정보.csv",
    "5. 재무제표 주석.csv",
    "3. 연결재무제표 주석.csv",
    "6. 배당에 관한 사항 등.csv",
    "3. 자본금 변동사항.csv",
    "VII. 주주에 관한 사항.csv",
    "IX. 계열회사 등에 관한 사항.csv",
    "XI. 그 밖에 투자자 보호를 위하여 필요한 사항.csv",
    "III. 재무에 관한 사항.csv",
]
medium_list = [
    "4. 주식의 총수 등.csv",
    "5. 의결권 현황.csv",
    "1. 이사회에 관한 사항.csv",
    "VI. 이사회 등 회사의 기관에 관한 사항.csv",
    "IV. 이사의 경영진단 및 분석의견.csv",
    "2. 임원의 보수 등.csv",
]

###
summary_system = "당신은 금융 전문가입니다. 주어진 금융 보고서 각각에 대해, step-by-step으로 요약하세요."

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 2048

import ast
import os

from sub_func import fin_statement_info # ticker, year, quarter -> df
from sub_func import reports_info # ticker, year, quarter -> df
from sub_func import corp_rel_news_info # ticker, year, start_date, end_date -> df
from sub_func import intl_news_info # year, start_date, end_date -> df
from sub_func import macro_econ_info # year, start_date, end_date -> dict
from sub_func import stock_price_info # ticker, start_date, end_date -> df
from sub_func import index_price_info # sector, start_date, end_date -> df
from sub_func import pattern_info # ticker, date -> df
from sub_func import sector_analysis_info # sector, year, quarter -> dict

from sub_func import to_GPT

from sub_func import preprocess_financial_text

def price_estimation_info():
    return None 

def current_portfolio_info():
    return None

def portfolio_analysis_info():
    return None

# ================================================================================================================ #

def get_ordered_report_dict(ticker, year, quarter):
    """
    중요도에 따라 보고서를 분류한 dict 반환
    """

    df = reports_info(ticker, year, quarter)
    report_importance_dict = {
        'High': {},
        'Medium': {}
    }

    csv_list = list(df.keys())
    for csv in csv_list:
        if csv in high_list:
            report_importance_dict['High'][csv] = ast.literal_eval(df[csv][0])[0]
        elif csv in medium_list:
            report_importance_dict['Medium'][csv] = ast.literal_eval(df[csv][0])[0]
        else:
            pass

    return report_importance_dict

def summarize_text(text):
    """
    금융 관련 문장 요약 모델
    """
    inputs = ["summarize: " + f"{preprocess_financial_text(text)}"]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=False, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=300)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    result = nltk.sent_tokenize(decoded_output.strip())[0]

    return result

def sliding_window_summary(text, max_chunk_len=1000, overlap_len=100):
    """
    긴 텍스트를 분할해 요약하고 결과를 재결합
    """
    # 텍스트 분할
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []

    # 슬라이딩 윈도우 방식으로 분할
    for sentence in sentences:
        if len(" ".join(current_chunk + [sentence])) <= max_chunk_len:
            current_chunk.append(sentence)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = sentences[
                max(0, len(current_chunk) - overlap_len):]  # 중첩된 부분 유지

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # 요약 결과 생성
    summaries = [summarize_text(chunk) for chunk in chunks]
    final_summary = " ".join(summaries)
    
    return final_summary

[nltk_data] Downloading package punkt to /Users/yeonsuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
target_year = '2020'
target_quarter = 'Q2'

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

fin_statement_dict = {}
fin_report_dict = {}
tickers_list = os.listdir('../store_data/raw/market_data/price/')

for i in range(len(tickers_list)):
    try:
        target_ticker = tickers_list[i]

        # 한 분기 이전 정보 찾기 (ex. 만약 지금이 Q3라면 Q2의 정보를 가져와야 함. 만약 Q1이라면 이전 년도의 Q4를 가져와야 함.)
        if target_quarter == 'Q1':
            target_year = f"{int(target_year)}-1"

        search_quarter_idx = (quarter_list.index(target_quarter) + 1) % 4
        search_quarter = quarter_list[search_quarter_idx]

        # 재무제표 분석 정보 가져와서 담아두기
        fin_statement = fin_statement_info(target_ticker, target_year, search_quarter)
        fin_statement_dict[target_ticker] = fin_statement

        # 재무보고서 가져와서 담아두기
        fin_report = get_ordered_report_dict(target_ticker, target_year, target_quarter)
        report_txt = ""
        for report in fin_report['Medium']:
            report_txt += f"{preprocess_financial_text(fin_report['Medium'][report])}"

        # response = to_GPT(summary_system, report_txt)['choices'][0]['message']['content']
        response = report_txt

        fin_report['Medium'] = ''
        fin_report['Medium'] = response
        fin_report_dict[target_ticker] = fin_report

    except:
        continue

재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/013700/_013700_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/013700/_013700_재무제표 ().csv'
013700의 이전 년도 재무 데이터를 불러올 수 없습니다.
013700의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/058860'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/058860'
058860의 이전 년도 재무 데이터를 불러올 수 없습니다.
058860의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/

In [11]:
report_txt

"{'cleaned_text': ' 2. 임원의 보수 등 <이사감사 전체의 보수현황> 1. 주주총회 승인금액 (단위 : 백만원) 구 분 인원수 주주총회 승인금액 비고 사내이사 및 사외이사 7 2,500  2. 보수지급금액 21. 이사감사 전체 (단위 : 백만원) 인원수 보수총액 1인당 평균보수액 비고 7 468 67  ※ 보수지급금액에는 2020년 3월 26일자로 퇴임 및 사임한 사외이사 2명의 보수가 포함되어 있습니다.※ 인원수는 2020년 6월 말 기준 인원수 입니다. 22. 유형별 (단위 : 백만원) 구 분 인원수 보수총액 1인당평균보수액 비고 등기이사(사외이사, 감사위원회 위원 제외) 3 396 132  사외이사(감사위원회 위원 제외) 1 24 24  감사위원회 위원 3 48 16  감사     3. 이사의 보수지급 기준  등기이사(사외이사, 감사위원회 위원 제외): 주주총회승인금액 내에서 이사회가 결정  사외이사(감사위원회 위원인 사외이사 제외): 주주총회승인금액 내에서 이사회가 결정  감사위원회 위원: 주주총회승인금액 내에서 이사회가 결정 <보수지급금액 5억원 이상인 이사감사의 개인별 보수현황> 1. 개인별 보수지급금액 (단위 : 원) 이름 직위 보수총액 보수총액에 포함되지 않는 보수         2020년 1월 1일부터 2020년 6월 30일까지 지급한 개인별 보수가 5억원 이상인 이사가 없어 해당사항 없습니다. 2. 산정기준 및 방법 (단위 : 백만원) 이름 보수의 종류 총액 산정기준 및 방법 근로소득 급여   상여   주식매수선택권 행사이익   기타 근로소득   퇴직소득   기타소득   <보수지급금액 5억원 이상 중 상위 5명의 개인별 보수현황> 1. 개인별 보수지급금액 (단위 : 백만원) 이름 직위 보수총액 보수총액에 포함되지 않는 보수         2020년 1월 1일부터 2020년 6월 30일까지 지급한 개인별 보수가 5억원 이상인 이사가 없어 해당사항 없습니다. 2. 산정기준 및 방법 (단위 : 백만원) 이름 보수의 종류 총액 산정기

In [3]:
summary_target_dict = fin_report['Medium']

for summary in summary_target_dict:
    target_text = summary_target_dict[summary]
    summarized = sliding_window_summary(target_text)

    summary_target_dict[summary] = ''
    summary_target_dict[summary] = summarized
    print(summarized)

보수 지급금액에는 2020년 1월 1일부터 2020년 6월 30일까지 지급한 개인별 보수가 5억 원 이상인 이사가 없어 해당사항 없다. 보수 지급금액에는 2020년 1월 1일부터 2020년 6월 30일까지 지급한 개인별 보수가 5억 원 이상인 이사가 없어 해당사항 없다.


KeyboardInterrupt: 

In [8]:
fin_report_dict

NameError: name 'fin_report_dict' is not defined